In [1]:
import pandas as pd
import numpy as np
from  graphviz import Source
from scipy import misc
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataframe = pd.DataFrame()
for x in range(1,11):
    dataframe_name = '../data/block_'+str(x)+'.csv'
    new_dataframe = pd.read_csv(dataframe_name)
    dataframe = pd.concat([dataframe,new_dataframe])

frame = dataframe

In [3]:
broken_columns = list(frame.columns[2:11])

for column in broken_columns:
    frame[column] = frame[column].apply(lambda x: np.NaN if x == '?' else x)
    frame[column] = frame[column].apply(lambda x: float(x) if type(x) == str else x)

In [4]:
frame.drop(['id_1','id_2','cmp_fname_c2','cmp_lname_c2'],axis=1,inplace=True)

In [5]:
def preparer_data(frame):
    frame["cmp_fname_c1"] = frame["cmp_fname_c1"].replace(np.NaN,0.000235404896421846)
    frame["cmp_lname_c1"] = frame["cmp_lname_c1"].replace(np.NaN,2.68694413843136e-05)
    frame["cmp_sex"] = frame["cmp_sex"].replace(np.NaN,0.5)
    frame["cmp_bd"] = frame["cmp_bd"].replace(np.NaN,0.032258064516129)
    frame["cmp_bm"] = frame["cmp_bm"].replace(np.NaN,0.0833333333333333)
    frame["cmp_by"] = frame["cmp_by"].replace(np.NaN, 0.00943396226415094)
    frame["cmp_plz"] = frame["cmp_plz"].replace(np.NaN, 0.000422654268808115)
    
    return frame

frame = preparer_data(frame)

# Neural Network

Após todo o tratamento dos dados a solução utilizando redes neurais foi implementada nos passos a seguir.

Insere a coluna de resultados (**is_match**) na variável `y`.

In [6]:
y = frame['is_match']

Remove a coluna **is_match** para que após o treinamento o resultado gerado seja comparado com a coluna y (**is_match**) e insere na variável X.

In [7]:
X = frame.drop(['is_match'],axis=1)

Separando os valores de *test* e *train*. sendo 33% para teste.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Imprime a quantidade de colunas para dimensionar *hidden layer*.

In [9]:
X.axes

[Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                  8,      9,
             ...
             574903, 574904, 574905, 574906, 574907, 574908, 574909, 574910,
             574911, 574912],
            dtype='int64', length=5749132),
 Index(['cmp_fname_c1', 'cmp_lname_c1', 'cmp_sex', 'cmp_bd', 'cmp_bm', 'cmp_by',
        'cmp_plz'],
       dtype='object')]

Configurando as dimensões da rede neural para uma matriz 10x1.

In [10]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10,1), random_state=1)

Aplicando método `fit`, que tem por objetivo ajustar o modelo aos dados X e y.

In [11]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 1), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

Identificando os pesos da rede depois do treinamento.

In [12]:
print("Quantidade de camadas da rede", len(clf.coefs_))
print("Peso da camanda 0 =", len(clf.coefs_[0]))
print("Peso da camanda 1 =", len(clf.coefs_[1]))
print("Peso da camanda 2 =", len(clf.coefs_[2]))

Quantidade de camadas da rede 3
Peso da camanda 0 = 7
Peso da camanda 1 = 10
Peso da camanda 2 = 1


In [13]:
len(clf.intercepts_[0])

10

Executando o método `predict` a rede neural é usada para predizer o resultado dos dados. 

In [14]:
result = clf.predict(X_test)

## Teste

Matriz de confusão

In [15]:
print(confusion_matrix(y_test, result))

[[1890337       0]
 [   6877       0]]


Resultados:

In [16]:
print(classification_report(y_test, result))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00   1890337
       True       0.00      0.00      0.00      6877

avg / total       0.99      1.00      0.99   1897214



/home/arthur/.virtualenvs/1ml/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## K-fold

Para correlacionar os modelos, foi utilizado o método **k-fold** em que seu resultado foi utilizado para identificar o melhor algoritmo.

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
cv_result = cross_val_score(MLPClassifier(), X, y, cv=5)

O resultado alcançado com a utilização do modelo **Neural Network** foi `9999833018261599%`.

In [21]:
np.mean(cv_result)

0.9999833018261599